In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler,VectorIndexer, OneHotEncoder, StringIndexer
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, CountVectorizer, HashingTF, IDF
from pyspark.ml.linalg import Vector
from pyspark.ml.classification import RandomForestClassifier, RandomForestClassificationModel, MultilayerPerceptronClassifier, MultilayerPerceptronClassificationModel, LinearSVC, LinearSVCModel, LogisticRegression, LogisticRegressionModel
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import lower, length, size

In [ ]:
data_top_pre = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/top.csv", inferSchema=True, header=True)
data_top = data_top_pre.na.drop()
#res1 = data_top.withColumn('id', sparkf.monotonically_increasing_id())

data_bott_pre = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/bottom.csv", inferSchema=True, header=True) #13662
data_bott = data_bott_pre.na.drop()
#data_bott.show(truncate=False)
#res11 = data_bott.withColumn('id', sparkf.monotonically_increasing_id())

data_gnd_pre = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/gnd_top_bottom_pairs.csv", inferSchema=True, header=False)
data_gnd = data_gnd_pre.na.drop()

data_top.show()
data_bott.show()
data_gnd.show()
joined_bott_top_aswr
joined_bott_top_ques
pre_dl_dataset
("./pre_dl_dataset.csv")


In [ ]:
#ALL
# Prepare data
data = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/pre_dl_dataset_textt.csv",inferSchema=True, header=True)

feature_prep = data.select(lower(data["bot_Cate"]).alias("bot_Cate"), lower(data["top_Cate"]).alias("top_Cate"),lower(data["bot_Desc"]).alias("bot_Desc"), lower(data["top_Desc"]).alias("top_Desc"), (data["top_id"]), (data["bot_id"]), "tag")

# Preprocessing and feature engineering
#Cate 2 Cate
feature_prep = RegexTokenizer(inputCol="bot_Cate", outputCol="words", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$

stopwordList = ["null", "Null", '', ' '] 
stopwordList.extend(StopWordsRemover().getStopWords())
stopwordList = list(set(stopwordList))

feature_prep = StopWordsRemover(inputCol='words',outputCol='stop_words_removed', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed", outputCol="tf", numFeatures=5000).transform(feature_prep)
feature_prep = IDF(inputCol="tf", outputCol="tf_idf").fit(feature_prep).transform(feature_prep)

feature_prep = RegexTokenizer(inputCol="top_Cate", outputCol="words2", pattern=r"^[a-z]+$").transform(feature_prep) 
feature_prep = StopWordsRemover(inputCol='words2',outputCol='stop_words_removed2', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed2", outputCol="tf2", numFeatures=5000).transform(feature_prep)
feature_prep = IDF(inputCol="tf2", outputCol="tf_idf2").fit(feature_prep).transform(feature_prep)
#Desc 2 Desc 

feature_prep = RegexTokenizer(inputCol="bot_Desc", outputCol="words3", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$
feature_prep = StopWordsRemover(inputCol='words3',outputCol='stop_words_removed3', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed3", outputCol="tf3", numFeatures=5000).transform(feature_prep)
feature_prep = IDF(inputCol="tf3", outputCol="tf_idf3").fit(feature_prep).transform(feature_prep)

feature_prep = RegexTokenizer(inputCol="top_Desc", outputCol="words4", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$
feature_prep = StopWordsRemover(inputCol='words4',outputCol='stop_words_removed4', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed4", outputCol="tf4", numFeatures=5000).transform(feature_prep)
feature_prep = IDF(inputCol="tf4", outputCol="tf_idf4").fit(feature_prep).transform(feature_prep)


feature_prep = StringIndexer(inputCol='tag',outputCol='label_indexed').fit(feature_prep).transform(feature_prep)
feature_prep = VectorAssembler(inputCols=["top_id", "bot_id","tf_idf", "tf_idf2","tf_idf3", "tf_idf4"], outputCol="features", handleInvalid="skip").transform(feature_prep)
final_data = feature_prep.select("label_indexed", "features")

# Split data into train and test sets
train_data, test_data = final_data.randomSplit([0.7,0.3])


# Model training
classifier = LogisticRegression(featuresCol="features", labelCol="label_indexed", regParam=0.1, elasticNetParam=1.0, family="multinomial")
classifier = RandomForestClassifier(featuresCol="features", labelCol="label_indexed", numTrees=500, maxDepth=30)
# 200 /25
# 1000/25
model = classifier.fit(train_data)

# Transform the test data using the model to get predictions
predicted_test_data = model.transform(test_data)

evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='label_indexed', predictionCol='prediction', metricName='accuracy')
print("Accuracy: {}", evaluator_accuracy.evaluate(predicted_test_data)) 

# Save the model
model.save('hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/ALLIN_classifier_model')

In [ ]:
#Cate 2 Cate
# Prepare data
data = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/pre_dl_dataset_text.csv",inferSchema=True, header=True)
# Preprocessing and feature engineering
feature_prep = data.select(lower(data["bot_Cate"]).alias("bot_Cate"), lower(data["top_Cate"]).alias("top_Cate"), "tag")

feature_prep = RegexTokenizer(inputCol="bot_Cate", outputCol="words", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$

stopwordList = ["null", "Null", '', ' '] 
stopwordList.extend(StopWordsRemover().getStopWords())
stopwordList = list(set(stopwordList))

feature_prep = StopWordsRemover(inputCol='words',outputCol='stop_words_removed', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed", outputCol="tf", numFeatures=5000).transform(feature_prep)
feature_prep = IDF(inputCol="tf", outputCol="tf_idf").fit(feature_prep).transform(feature_prep)


feature_prep = RegexTokenizer(inputCol="top_Cate", outputCol="words2", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$
feature_prep = StopWordsRemover(inputCol='words2',outputCol='stop_words_removed2', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed2", outputCol="tf2", numFeatures=5000).transform(feature_prep)
feature_prep = IDF(inputCol="tf2", outputCol="tf_idf2").fit(feature_prep).transform(feature_prep)
feature_prep = StringIndexer(inputCol='tag',outputCol='label_indexed').fit(feature_prep).transform(feature_prep)
feature_prep = VectorAssembler(inputCols=["tf_idf", "tf_idf2"], outputCol="features").transform(feature_prep)

final_data = feature_prep.select("label_indexed", "features")

# Split data into train and test sets
train_data, test_data = final_data.randomSplit([0.7,0.3])

#==============================================================================
# Model training'''
classifier = RandomForestClassifier(featuresCol="features", labelCol="label_indexed", numTrees=500, maxDepth=30)
# 200 /25
classifier = LogisticRegression(featuresCol="features", labelCol="label_indexed", regParam=0.1, elasticNetParam=1.0, family="multinomial")

#pyspark.ml.classification.NaiveBayes(featuresCol='features', labelCol='label', predictionCol='prediction', probabilityCol='probability', rawPredictionCol='rawPrediction', smoothing=1.0, modelType='multinomial', thresholds=None, weightCol=None)

model = classifier.fit(train_data)


# Transform the test data using the model to get predictions
predicted_test_data = model.transform(test_data)

evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='label_indexed', predictionCol='prediction', metricName='accuracy')
print("Accuracy: {}", evaluator_accuracy.evaluate(predicted_test_data)) 

# Save the model
model.save('hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/Cate_classifier_model_LR')


In [ ]:
#Desc 2 Desc
# Prepare data
data = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/pre_dl_dataset_text.csv",inferSchema=True, header=True)
# Preprocessing and feature engineering
feature_prep = data.select(lower(data["bot_Desc"]).alias("bot_Desc"), lower(data["top_Desc"]).alias("top_Desc"), "tag")

feature_prep = RegexTokenizer(inputCol="bot_Desc", outputCol="words", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$

stopwordList = ["null", "Null", '', ' '] 
stopwordList.extend(StopWordsRemover().getStopWords())
stopwordList = list(set(stopwordList))

feature_prep = StopWordsRemover(inputCol='words',outputCol='stop_words_removed', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed", outputCol="tf", numFeatures=5000).transform(feature_prep)
feature_prep = IDF(inputCol="tf", outputCol="tf_idf").fit(feature_prep).transform(feature_prep)

feature_prep = RegexTokenizer(inputCol="top_Desc", outputCol="words2", pattern=r"^[a-z]+$").transform(feature_prep) #^[a-z]+$
feature_prep = StopWordsRemover(inputCol='words2',outputCol='stop_words_removed2', stopWords=stopwordList).transform(feature_prep)
feature_prep = HashingTF(inputCol="stop_words_removed2", outputCol="tf2", numFeatures=5000).transform(feature_prep)
feature_prep = IDF(inputCol="tf2", outputCol="tf_idf2").fit(feature_prep).transform(feature_prep)

feature_prep = StringIndexer(inputCol='tag',outputCol='label_indexed').fit(feature_prep).transform(feature_prep)
feature_prep = VectorAssembler(inputCols=["tf_idf", "tf_idf2"], outputCol="features").transform(feature_prep)
final_data = feature_prep.select("label_indexed", "features")

# Split data into train and test sets
train_data, test_data = final_data.randomSplit([0.7,0.3])

# Model training
classifier = RandomForestClassifier(featuresCol="features", labelCol="label_indexed", numTrees=800, maxDepth=25)
#500 / 30 
#1000/25
classifier = LogisticRegression(featuresCol="features", labelCol="label_indexed", regParam=0.1, elasticNetParam=1.0, family="multinomial")

model = classifier.fit(train_data)

# Transform the test data using the model to get predictions
predicted_test_data = model.transform(test_data)

evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='label_indexed', predictionCol='prediction', metricName='accuracy')
print("Accuracy: {}", evaluator_accuracy.evaluate(predicted_test_data)) 

# Save the model
model.save("hdfs://devenv/user/spark/spark_mllib_101/spam_detection/Desc_classifier_model")


In [ ]:
#ID 2 ID
# Prepare data
data = spark.read.csv("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/pre_dl_dataset_textt.csv",inferSchema=True, header=True)
# Preprocessing and feature engineering
feature_prep = data.select((data["top_id"]), (data["bot_id"]), "tag")
feature_prep = VectorAssembler(inputCols=["top_id", "bot_id"], outputCol="features", handleInvalid="skip").transform(feature_prep)
final_data = feature_prep.select("tag", "features")

# Split data into train and test sets
train_data, test_data = final_data.randomSplit([0.7,0.3])

# Model training
classifier = LogisticRegression(featuresCol="features", labelCol="tag", regParam=0.1, elasticNetParam=1.0)
#classifier = RandomForestClassifier(featuresCol="features", labelCol="tag", numTrees=200, maxDepth=25)


model = classifier.fit(train_data)
# Transform the test data using the model to get predictions
predicted_test_data = model.transform(test_data)

evaluator_accuracy = MulticlassClassificationEvaluator(labelCol='tag', predictionCol='prediction', metricName='accuracy')
print("Accuracy: {}", evaluator_accuracy.evaluate(predicted_test_data)) 

# Save the model
model.save("hdfs://master.tibame/user/heart/project/dataset/FashionVCdata/id_classifier_model")